In [1]:
# Importing the necessary libraries

import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score
import pandas as pd

In [2]:
from zipfile import ZipFile
import os

zip_file_name = '/content/DUCK-Net-main.zip'
model_folder_path = '/content/DUCK-Net-main'

if not os.path.exists(model_folder_path):
    os.makedirs(model_folder_path)

with ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(model_folder_path)

In [3]:
import sys
sys.path.append('/content/DUCK-Net-main/CustomLayers')
import ConvBlock2D
sys.path.append('/content/DUCK-Net-main/ModelArchitecture')
import DiceLoss
from DiceLoss import dice_metric_loss
import DUCK_Net
sys.path.append('/content/DUCK-Net-main/ImageLoader')
import ImageLoader2D

In [4]:
# Setting the model parameters

img_size = 352
dataset_type = 'kvasir' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 34 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

min_loss_for_saving = 0.2

In [7]:
from zipfile import ZipFile
import os

zip_file_name = '/content/kvasir-seg.zip'
dataset_path = '/content/'

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

with ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

dataset_path +='/Kvasir-SEG/'

In [6]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive/')
saved_model_path = Path('/content/drive/My Drive/Computer Vision/DuckNet34 Cvc-ColonDb Tf Model')


Mounted at /content/drive/


In [8]:
# Loading the data

X, Y = ImageLoader2D.load_data(img_size, img_size, -1, 'kvasir', dataset_path)

Resizing training images and masks: 1000


1000it [18:03,  1.08s/it]


In [9]:
# Load the model

print("Loading the model")

model = tf.keras.models.load_model(saved_model_path, custom_objects={'dice_metric_loss':dice_metric_loss})

Loading the model


In [10]:
prediction_test = model.predict(X, batch_size=4)
print("Predictions done")


dice_test = f1_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
print("Dice finished")


miou_test = jaccard_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
print("Miou finished")

# Save the DataFrame to an Excel file
dice_scores = []
miou_scores = []

for i in range(len(Y)):
    dice_score = f1_score(np.ndarray.flatten(np.array(Y[i], dtype=bool)),
                          np.ndarray.flatten(prediction_test[i] > 0.5))
    miou_score = jaccard_score(np.ndarray.flatten(np.array(Y[i], dtype=bool)),
                                np.ndarray.flatten(prediction_test[i] > 0.5))
    dice_scores.append(dice_score)
    miou_scores.append(miou_score)

data = {'Image': range(len(Y)), 'Dice': dice_scores, 'MIOU': miou_scores}
image_metrics_df = pd.DataFrame(data)

image_metrics_df.to_excel("image-metrics-DUCKNet-train-colondb-test-kvasir..xlsx", index=False)

print("Excel file with image-wise metrics generated successfully.")


precision_test = precision_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
print("Precision finished")


recall_test = recall_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
print("Recall finished")


accuracy_test = accuracy_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
print("Accuracy finished")

final_file = "results-DUCKNet-train-colondb-test-kvasir.txt"
print(final_file)


with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write(' dice_test: ' + str(dice_test) + '\n\n')
    f.write(' miou_test: ' + str(miou_test) + '\n\n')
    f.write(' precision_test: ' + str(precision_test) + '\n\n')
    f.write(' recall_test: ' + str(recall_test) + '\n\n')
    f.write(' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')

250/250 [==============================] - 10610s 42s/step
Predictions done
Dice finished
Miou finished
Excel file with image-wise metrics generated successfully.
Precision finished
Recall finished
Accuracy finished
results-DUCKNet-train-colondb-test-kvasir.txt
File done
